### Decide which hand-crafted rules are meaning-preserving and thus safe to include in the reward model (RM) or PPO training.

- look into an LLM that provides a targeted German support:
  - "xlm-roberta-base"
  - "dbmdz/bert-base-german-uncased"
  - deepset/gbert-base
  - bert-base-german-dbmdz-uncased

- simplification score to be:
  - the rule-compliance tracker
  - inserting SARI as well would be too 'simple minded'

possible simplification score combination
- combine 
- reward = alpha * simplification_score + beta * bert_score
- alpha, beta can be tuned depending on priorities (which score is more critical?)

In [27]:
# pip install torch torchvision transformers
# pip install bert-score

In [94]:
from bert_score import score
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import pandas as pd
import re

In [153]:
input_path = "master_data/0_original/all.txt"
output_path = "master_data/3_simplified/all_simplified_plain.txt"
#log_path = "simplification_logs/all_parsed_log_2025-09-13_23-31-26.csv"
log_path = "simplification_logs/all_parsed_log_2025-09-14_12-38-08.csv"

In [154]:
df = pd.read_csv(log_path)

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112715 entries, 0 to 112714
Data columns (total 7 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   uid                        112715 non-null  int64 
 1   original                   112710 non-null  object
 2   initial_original_sentence  112715 non-null  object
 3   rule                       112715 non-null  object
 4   applied                    112715 non-null  bool  
 5   simplified                 112654 non-null  object
 6   doc_name                   112715 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 5.3+ MB


#### There are non-null rows in simplified, identified to come from word_to_number() vconversion. They need to be filtered out.

In [156]:
df = df.dropna(how='any', axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112654 entries, 0 to 112714
Data columns (total 7 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   uid                        112654 non-null  int64 
 1   original                   112654 non-null  object
 2   initial_original_sentence  112654 non-null  object
 3   rule                       112654 non-null  object
 4   applied                    112654 non-null  bool  
 5   simplified                 112654 non-null  object
 6   doc_name                   112654 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 6.1+ MB


In [157]:
df.head(10)

,uid,original,initial_original_sentence,rule,applied,simplified,doc_name
0,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,clean_punctuation,False,Der Iran wird teilweise aus dem Atom-Abkommen ...,all_parsed.txt
1,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,rewrite_apposition,False,Der Iran wird teilweise aus dem Atom-Abkommen ...,all_parsed.txt
2,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,simplify_subordinate,False,Der Iran wird teilweise aus dem Atom-Abkommen ...,all_parsed.txt
3,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,convert_passive_to_active,False,Der Iran wird teilweise aus dem Atom-Abkommen ...,all_parsed.txt
4,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,normalize_verb_tense,True,Der Iran wird teilweise aus dem Atom-Abkommen ...,all_parsed.txt
5,2,Präsidentin,Brüssel Ursula von der Leyen ist die Präsident...,split_compound,True,Präsi·Dentin,all_parsed.txt
6,2,Brüssel Ursula von der Leyen ist die Präsi·Den...,Brüssel Ursula von der Leyen ist die Präsident...,clean_punctuation,False,Brüssel Ursula von der Leyen ist die Präsi·Den...,all_parsed.txt
7,2,Brüssel Ursula von der Leyen ist die Präsi·Den...,Brüssel Ursula von der Leyen ist die Präsident...,rewrite_apposition,False,Brüssel Ursula von der Leyen ist die Präsi·Den...,all_parsed.txt
8,2,Brüssel Ursula von der Leyen ist die Präsi·Den...,Brüssel Ursula von der Leyen ist die Präsident...,simplify_subordinate,False,Brüssel Ursula von der Leyen ist die Präsi·Den...,all_parsed.txt
9,2,Brüssel Ursula von der Leyen ist die Präsi·Den...,Brüssel Ursula von der Leyen ist die Präsident...,convert_passive_to_active,False,Brüssel Ursula von der Leyen ist die Präsi·Den...,all_parsed.txt


In [158]:
df.to_csv("master_data/output_assessment/all_simplifications.csv", index=False)

## Exploration

In [60]:
df_compound.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24784 entries, 5 to 112709
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         24784 non-null  int64 
 1   original    24784 non-null  object
 2   rule        24784 non-null  object
 3   applied     24784 non-null  bool  
 4   simplified  24783 non-null  object
 5   doc_name    24784 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 1.2+ MB


In [61]:
df_number = df[df["rule"] == "convert_word_to_number"]
df_number.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4258 entries, 12 to 112708
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         4258 non-null   int64 
 1   original    4258 non-null   object
 2   rule        4258 non-null   object
 3   applied     4258 non-null   bool  
 4   simplified  4203 non-null   object
 5   doc_name    4258 non-null   object
dtypes: bool(1), int64(1), object(4)
memory usage: 203.8+ KB


In [64]:
filtered_comp = df_compound[df_compound["applied"] == True]
filtered_comp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4964 entries, 5 to 112665
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         4964 non-null   int64 
 1   original    4964 non-null   object
 2   rule        4964 non-null   object
 3   applied     4964 non-null   bool  
 4   simplified  4963 non-null   object
 5   doc_name    4964 non-null   object
dtypes: bool(1), int64(1), object(4)
memory usage: 237.5+ KB


In [63]:
filtered_number = df_number[df_number["applied"] == True]
filtered_number.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1020 entries, 12 to 112708
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         1020 non-null   int64 
 1   original    1020 non-null   object
 2   rule        1020 non-null   object
 3   applied     1020 non-null   bool  
 4   simplified  965 non-null    object
 5   doc_name    1020 non-null   object
dtypes: bool(1), int64(1), object(4)
memory usage: 48.8+ KB


In [36]:
# def assess_rule_output(df):
#     results = []

#     for uid, group in df.groupby("uid"):
#         original = group["original"].iloc[0]              # the very first "original" sentence
#         simplified = group["simplified"].iloc[-1]         # the last simplification
#         applied_rules = group.loc[group["applied"] == True, "rule"].tolist()

#         results.append({
#             "uid": uid,
#             "original": original,
#             "simplified": simplified,
#             "applied_rules": applied_rules
#         })

#     return pd.DataFrame(results)

## Outdated approach to aggregate from applied=True approach

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112715 entries, 0 to 112714
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   uid         112715 non-null  int64 
 1   original    112710 non-null  object
 2   rule        112715 non-null  object
 3   applied     112715 non-null  bool  
 4   simplified  112654 non-null  object
 5   doc_name    112715 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 4.4+ MB


In [ ]:
#Filter out only applied rules
df_applied = df[df["applied"] == True]
df_applied.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15032 entries, 4 to 112708
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         15032 non-null  int64 
 1   original    15032 non-null  object
 2   rule        15032 non-null  object
 3   applied     15032 non-null  bool  
 4   simplified  14976 non-null  object
 5   doc_name    15032 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 719.3+ KB


In [73]:
df_applied_dropped.head(15)

,uid,original,rule,applied,simplified,doc_name
4,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,normalize_verb_tense,True,Der Iran wird teilweise aus dem Atom-Abkommen ...,all_parsed.txt
5,2,Präsidentin,split_compound,True,Präsi·Dentin,all_parsed.txt
12,3,erste,convert_word_to_number,True,1.,all_parsed.txt
25,4,Bis zum Jahr 2030 soll es in der Europäische U...,normalize_verb_tense,True,Bis zum Jahr 2030 soll es in der Europäische U...,all_parsed.txt
34,5,"Das ist sehr viel , denn in den letzten 29 Jah...",normalize_verb_tense,True,"Das hat ist sehr viel , denn in den letzten 29...",all_parsed.txt
48,7,Die Europäische Union-Kommission will vor alle...,normalize_verb_tense,True,Die Europäische Union-Kommission will vor alle...,all_parsed.txt
52,8,Lade-Stationen,split_compound,True,Lade·Stationen,all_parsed.txt
53,8,Elektro-Fahrzeuge,split_compound,True,Elektro·Fahrzeuge,all_parsed.txt
65,10,Union-Gesetze,split_compound,True,Union·Gesetze,all_parsed.txt
93,15,Corona-Virus,split_compound,True,Corona·Virus,all_parsed.txt


In [74]:
df_applied_dropped.to_csv("master_data/output_assessment/all_applied_rules.csv", index=False)

In [41]:
#OUTDATED code for the original simplification approach on original/complex sentences

# # Get the last applied simplification per sentence UID
# # (Assume rules are applied in order of appearance)
# last_applied_per_uid = df_applied.groupby("uid").tail(1)

# # Also get original sentences from any row (all identical for a UID)
# originals_per_uid = df.groupby("uid").first().reset_index()[["uid", "original"]]

# # Merge to get (original, final simplified) pairs
# final_pairs = pd.merge(originals_per_uid, last_applied_per_uid[["uid", "simplified"]], on="uid")

# # Extract all applied rules per UID (True only)
# # gives out UID and a second column of all applied rules according to uid

# applied_rules_per_uid = (
#     df[df["applied"] == True]
#     .groupby("uid")["rule"]
#     .apply(list)
#     .reset_index()
#     .rename(columns={"rule": "applied_rules"})
# )

# # Merge with the final_pairs (which already has original + final simplified)
# final_pairs_with_rules = pd.merge(final_pairs, applied_rules_per_uid, on="uid", how="left")

In [ ]:
# ### ====OUTDATED, was applied on applied=True ==== ###
# # Get the unique original sentences in the order of their first appearance
# #unique_originals = df_applied['original'].unique()
# grouped = df_applied_dropped.groupby('uid')

# processed_data = []

# #for sentence in unique_originals:
# for uid, group in grouped:
#         # Get all rows for the current original sentence
#         #group = df_applied[df_applied['original'] == sentence]
        
#         # Find all rules that were successfully applied for this group
#         applied_rules_list = group[group['applied'] == True]['rule'].tolist()
        
#         # We only want to include sentences where at least one rule was applied
#         if not applied_rules_list:
#             continue

#         # De-duplicate the list of rules while preserving order
#         unique_applied_rules = list(dict.fromkeys(applied_rules_list))

#         # Heuristic: The "main" original sentence is the longest one in the group
#         main_original_sentence = group.loc[group['original'].str.len().idxmax(), 'original']

#         # The final simplification is the 'simplified' text from the very last logged step
#         final_simplification_text = group.loc[group.index.max(), 'simplified']

#         # Alternative (if you want the very last simplification regardless of application):    
#         # The final simplification is the 'simplified' text from the very last entry in the group
#         #final_simplification_text = group['simplified'].iloc[-1]
        
#         # Append the structured data
#         processed_data.append({
#             'uid': uid,
#             'original_sentence': main_original_sentence,
#             'final_simplification': final_simplification_text,
#             'applied_rules': unique_applied_rules
#         })

# # Create the final DataFrame from our processed list
# result_df = pd.DataFrame(processed_data)

# Filter out and aggregate from simplification log

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   uid                        0 non-null      int64 
 1   original                   0 non-null      object
 2   initial_original_sentence  0 non-null      object
 3   rule                       0 non-null      object
 4   applied                    0 non-null      bool  
 5   simplified                 0 non-null      object
 6   doc_name                   0 non-null      object
dtypes: bool(1), int64(1), object(5)
memory usage: 0.0+ bytes


In [137]:
# Sort by UID to ensure chronological order of steps, reset index due to dropped rows
df = df.sort_values(by=['uid']).reset_index(drop=True)

# Group by the unique identifier for each original sentence
grouped_by_sentence = df.groupby('uid')

In [159]:
grouped = df.groupby('uid')

processed_data = []

#for sentence in unique_originals:
for uid, group in grouped:
        # Get all rows for the current original sentence
        #group = df_applied[df_applied['original'] == sentence]
        
        # Find all rules that were successfully applied for this group
        applied_rules_list = group[group['applied'] == True]['rule'].tolist()
        
        # We only want to include sentences where at least one rule was applied
        if not applied_rules_list:
            continue

        # De-duplicate the list of rules while preserving order
        unique_applied_rules = list(dict.fromkeys(applied_rules_list))

        # Heuristic: The "main" original sentence is the longest one in the group
        main_original_sentence = group.loc[group['original'].str.len().idxmax(), 'original']

        # The final simplification is the 'simplified' text from the very last logged step
        final_simplification_text = group.loc[group.index.max(), 'simplified']

        # Alternative (if you want the very last simplification regardless of application):    
        # The final simplification is the 'simplified' text from the very last entry in the group
        #final_simplification_text = group['simplified'].iloc[-1]
        
        # Append the structured data
        processed_data.append({
            'uid': uid,
            'original_sentence': main_original_sentence,
            'final_simplification': final_simplification_text,
            'applied_rules': unique_applied_rules
        })

# Create the final DataFrame from our processed list
result_df = pd.DataFrame(processed_data)

In [160]:
result_df.head(15)

,uid,original_sentence,final_simplification,applied_rules
0,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,[normalize_verb_tense]
1,2,Brüssel Ursula von der Leyen ist die Präsi·Den...,Brüssel Ursula von der Leyen ist die Präsi·Den...,[split_compound]
2,3,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,[convert_word_to_number]
3,4,Bis zum Jahr 2030 soll es in der Europäische U...,Bis zum Jahr 2030 soll es in der Europäische U...,[normalize_verb_tense]
4,5,"Das ist sehr viel , denn in den letzten 29 Jah...","Das hat ist sehr viel , denn in den letzten 29...",[normalize_verb_tense]
5,7,Die Europäische Union-Kommission will vor alle...,Die Europäische Union-Kommission will vor alle...,[normalize_verb_tense]
6,8,Dazu gehören zum Beispiel die Renovierung von ...,Dazu gehören zum Beispiel die Renovierung von ...,[split_compound]
7,10,Sie schlägt zum Beispiel Europäische Union·Ges...,Sie schlägt zum Beispiel Europäische Union·Ges...,[split_compound]
8,15,Die Maßnahmen gegen den Corona·Virus sind sehr...,Die Maßnahmen gegen den Corona·Virus sind sehr...,[split_compound]
9,17,Vor allem in Europa und in den USA schrumpft d...,Dann Vor allem in Europa und in den USA schrum...,[simplify_subordinate]


In [136]:
# Create the final DataFrame from the processed list
result_df = pd.DataFrame(processed_data)

# Display the first few rows of the result
result_df.head(15)


,uid,original_sentence,final_simplification,applied_rules
0,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,[normalize_verb_tense]
1,2,Brüssel Ursula von der Leyen ist die Präsident...,Präsi·Dentin,[split_compound]
2,3,Am Mittwoch hat sie ihre erste Rede zur Lage d...,1.,[convert_word_to_number]
3,4,Bis zum Jahr 2030 soll es in der Europäische U...,Bis zum Jahr 2030 soll es in der Europäische U...,[normalize_verb_tense]
4,5,"Das ist sehr viel , denn in den letzten 29 Jah...","Das hat ist sehr viel , denn in den letzten 29...",[normalize_verb_tense]
5,6,Für das neue Ziel bleiben aber weniger als 10 ...,Für das neue Ziel bleiben aber weniger als 10 ...,[]
6,7,Die Europäische Union-Kommission will vor alle...,Die Europäische Union-Kommission will vor alle...,[normalize_verb_tense]
7,8,Dazu gehören zum Beispiel die Renovierung von ...,Elektro·Fahrzeuge,[split_compound]
8,8,Dazu gehören zum Beispiel die Renovierung von ...,Lade·Stationen,[split_compound]
9,9,Die Europäische Union-Kommission ist ein wicht...,Die Europäische Union-Kommission ist ein wicht...,[]


In [126]:
# Sort the final result by UID to approximate the original file order
result_df = result_df.sort_values(by='uid').reset_index(drop=True)
result_df

,uid,original_sentence,final_simplification,applied_rules
0,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,[normalize_verb_tense]
1,2,Brüssel Ursula von der Leyen ist die Präsident...,Brüssel Ursula von der Leyen ist die Präsi·Den...,[split_compound]
2,3,Am Mittwoch hat sie ihre erste Rede zur Lage d...,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,[convert_word_to_number]
3,4,Bis zum Jahr 2030 soll es in der Europäische U...,Bis zum Jahr 2030 soll es in der Europäische U...,[normalize_verb_tense]
4,5,"Das ist sehr viel , denn in den letzten 29 Jah...","Das hat ist sehr viel , denn in den letzten 29...",[normalize_verb_tense]
...,...,...,...,...
16540,16542,Dann wird Hütter der neue Trainer von Borussia...,Dann wird Hütter der neue Trainer von Borussia...,[]
16541,16543,Er unterschreibt dort einen Vertrag bis zum Jahr,Er unterschreibt dort einen Vertrag bis zum Jahr,[]
16542,16544,2024.,2024,[convert_word_to_number]
16543,16545,Für Hütter muss Mönchengladbach eine hohe Ablö...,Für Hütter muss Mönchengladbach eine hohe Ablö...,[normalize_verb_tense]


In [127]:
result_df.head()

,uid,original_sentence,final_simplification,applied_rules
0,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,[normalize_verb_tense]
1,2,Brüssel Ursula von der Leyen ist die Präsident...,Brüssel Ursula von der Leyen ist die Präsi·Den...,[split_compound]
2,3,Am Mittwoch hat sie ihre erste Rede zur Lage d...,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,[convert_word_to_number]
3,4,Bis zum Jahr 2030 soll es in der Europäische U...,Bis zum Jahr 2030 soll es in der Europäische U...,[normalize_verb_tense]
4,5,"Das ist sehr viel , denn in den letzten 29 Jah...","Das hat ist sehr viel , denn in den letzten 29...",[normalize_verb_tense]


In [161]:
df_cleanup = result_df.copy()

In [162]:
df_cleanup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11107 entries, 0 to 11106
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   uid                   11107 non-null  int64 
 1   original_sentence     11107 non-null  object
 2   final_simplification  11107 non-null  object
 3   applied_rules         11107 non-null  object
dtypes: int64(1), object(3)
memory usage: 347.2+ KB


In [163]:

df_cleanup.columns = df_cleanup.columns.str.strip() # This removes leading/trailing spaces from each column name

def clean_all_whitespace(sentence):
  """
  Replaces multiple spaces inside a string with a single space,
  and then strips leading/trailing whitespace.
  """
  # 0: If the input is not a string, return it as is
  if not isinstance(sentence, str):
      return sentence
  # 1: Clean up all internal whitespace first.
  sentence = re.sub(r'\s+', ' ', sentence).strip()
  # 2: Strip whitespace from the beginning and end
  sentence = re.sub(r'\s+([.,:;?!])', r'\1', sentence)
  return sentence

columns_to_clean = ['original_sentence', 'final_simplification']

print(f"Attempting to strip whitespace from columns: {', '.join(columns_to_clean)}")

# Loop through the identified columns and apply the strip() method
for col in columns_to_clean:
  if col in df_cleanup.columns and df_cleanup[col].dtype == 'object':
    print(f"Cleaning column: '{col}'...")
    # Apply our new, more powerful cleaning function to each sentence in the column
    df_cleanup[col] = df_cleanup[col].apply(clean_all_whitespace)
  else:
    print(f"Column '{col}' not found or is not a text column.")

Attempting to strip whitespace from columns: original_sentence, final_simplification
Cleaning column: 'original_sentence'...
Cleaning column: 'final_simplification'...


In [164]:
print(df_cleanup.head().to_markdown(index=False))

|   uid | original_sentence                                                                              | final_simplification                                                                           | applied_rules              |
|------:|:-----------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------|:---------------------------|
|     1 | Der Iran wird teilweise aus dem Atom-Abkommen aussteigen.                                      | Der Iran wird teilweise aus dem Atom-Abkommen aussteigen.                                      | ['normalize_verb_tense']   |
|     2 | Brüssel Ursula von der Leyen ist die Präsi·Dentin von der Europäische Union-Kommission.        | Brüssel Ursula von der Leyen ist die Präsi·Dentin von der Europäische Union-Kommission.        | ['split_compound']         |
|     3 | Am Mittwoch hat sie ihre 1. Rede zur Lage der Europäische 

In [165]:
df_cleanup.head(20)

,uid,original_sentence,final_simplification,applied_rules
0,1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,[normalize_verb_tense]
1,2,Brüssel Ursula von der Leyen ist die Präsi·Den...,Brüssel Ursula von der Leyen ist die Präsi·Den...,[split_compound]
2,3,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,[convert_word_to_number]
3,4,Bis zum Jahr 2030 soll es in der Europäische U...,Bis zum Jahr 2030 soll es in der Europäische U...,[normalize_verb_tense]
4,5,"Das ist sehr viel, denn in den letzten 29 Jahr...","Das hat ist sehr viel, denn in den letzten 29 ...",[normalize_verb_tense]
5,7,Die Europäische Union-Kommission will vor alle...,Die Europäische Union-Kommission will vor alle...,[normalize_verb_tense]
6,8,Dazu gehören zum Beispiel die Renovierung von ...,Dazu gehören zum Beispiel die Renovierung von ...,[split_compound]
7,10,Sie schlägt zum Beispiel Europäische Union·Ges...,Sie schlägt zum Beispiel Europäische Union·Ges...,[split_compound]
8,15,Die Maßnahmen gegen den Corona·Virus sind sehr...,Die Maßnahmen gegen den Corona·Virus sind sehr...,[split_compound]
9,17,Vor allem in Europa und in den USA schrumpft d...,Dann Vor allem in Europa und in den USA schrum...,[simplify_subordinate]


In [168]:
output_filename = 'master_data/output_assessment/ordered_simplifications_with_rules_clean_FINAL.csv'
df_cleanup.to_csv(output_filename, index=False)

In [171]:
print(f"\nFinally saved the final, ordered file: '{output_filename}'")
print("\nHere is a preview of the new format:")
print(df_cleanup.head().to_markdown(index=False))


Finally saved the final, ordered file: 'master_data/output_assessment/ordered_simplifications_with_rules.csv'

Here is a preview of the new format:
|   uid | original_sentence                                                                              | final_simplification                                                                           | applied_rules              |
|------:|:-----------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------|:---------------------------|
|     1 | Der Iran wird teilweise aus dem Atom-Abkommen aussteigen.                                      | Der Iran wird teilweise aus dem Atom-Abkommen aussteigen.                                      | ['normalize_verb_tense']   |
|     2 | Brüssel Ursula von der Leyen ist die Präsi·Dentin von der Europäische Union-Kommission.        | Brüssel Ursula von der Leyen ist die Präsi·De

In [180]:
#only keel original_sentence and final_simplification
final_pairs = df_cleanup[['original_sentence', 'final_simplification']]
final_pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11107 entries, 0 to 11106
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentence     11107 non-null  object
 1   final_simplification  11107 non-null  object
dtypes: object(2)
memory usage: 173.7+ KB


In [181]:
final_pairs.to_csv("master_data/output_assessment/final_simplified_pairs_cleaned.csv", index=False)

# Assess the Performance using roberta (optional)

In [ ]:
# Load your exported sentence pairs
df = pd.read_csv("final_simplified_pairs.csv")

originals = df["original"].tolist()
simplifieds = df["simplified"].tolist()

# Compute BERTScore using German-specific model
P, R, F1 = score(simplifieds, originals, model_type="xlm-roberta-base", lang="de")

# Add scores back to dataframe
df["bertscore_f1"] = F1.tolist()

# Save the results
df.to_csv("bert_score_results.csv", index=False)
print("Done! Results saved to 'bert_score_results.csv'")

In [ ]:
# calculate with the csv files that includes applied rules
from collections import defaultdict

# Load final output with applied rules
df_rules = pd.read_csv("final_simplified_pairs_with_rules.csv")  # <- export this first from our current state

# Group sentence pairs by rule
rule_to_pairs = defaultdict(list)

for _, row in df_rules.iterrows():
    if pd.isna(row["applied_rules"]):
        continue
    rules = eval(row["applied_rules"]) if isinstance(row["applied_rules"], str) else row["applied_rules"]
    for rule in rules:
        rule_to_pairs[rule].append((row["original"], row["simplified"]))

# Compute average BERTScore-F1 per rule
rule_to_f1 = {}

for rule, pairs in rule_to_pairs.items():
    o, s = zip(*pairs)
    _, _, F1 = score(s, o, model_type="xlm-roberta-base", lang="de") #bert-base-german-dbmdz-uncased #
    rule_to_f1[rule] = sum(F1.tolist()) / len(F1)

# Print results
print("Average BERTScore-F1 per rule:")
for rule, f1 in sorted(rule_to_f1.items(), key=lambda x: x[1]):
    print(f"{rule}: {f1:.4f}")

In [ ]:
# #Example calc

# # Original vs. Simplified sentences
# originals = ["Der Hund läuft schnell zur Tür."]
# simplifieds = ["Der Hund rennt zur Tür."]

# # Compute BERTScore using a German or multilingual model
# P, R, F1 = score(simplifieds, originals, lang="de", model_type="bert-base-multilingual-cased")

# print(f"Precision: {P.mean().item():.4f}")
# print(f"Recall: {R.mean().item():.4f}")
# print(f"F1: {F1.mean().item():.4f}")
